In [1]:
#!/gpfs01/bartels/user/vplikat/anaconda3/bin/python

##########
# HEADER #
##########
# The data for this script is derived from an fMRI experiment in which
# subjects viewed different videos. Either magic, control or surprise videos. 
# The experiment was devided into 3 blocks. Each block consisted of 4 
# experimental runs. In each run subjects viewed 24 videos (each video is 
# considered a trial).
# The videos in each block were associated with one object (Balls, Cards and 
# Sticks) and there were 3 magic effects (Appear, Change and Vanish). For each
# magic effect and object there are two trick versions (i.e. Appear1, Appear2,
# Change1,...). This resulted in 6 magic videos per object = 18 magic videos 
# and for every magic video there was a corresponding control video showing
# the same movements without the magical effect. Additionally per object there
# were 3 surprise videos showing unusual surprising actions performed with the
# objects (e.g. eating a playing card).
# After the second run in each block the underlying method behind each magic 
# trick was presented.

#                       TIME
#   ---------------------------------->
#   OBJECT1 R1  R2  Revelation  R3  R4  |
#   OBJECT2 R1  R2  Revelation  R3  R4  |   TIME
#   OBJECT3 R1  R2  Revelation  R3  R4  v

# RUNS: 2*Appear1 Magic 2*Appear2 Magic 2*Appear1 Control 2*Appear2 Control
#       2*Vanish1 Magic 2*Vanish2 Magic 2*Vanish1 Control 2*Vanish2 Control
#       2*Change1 Magic 2*Change2 Magic 2*Change1 Control 2*Change2 Control
#       2*Surprise1     2*Surprise2     2*Surprise13
#       = 24 Videos

# The aim of the experiment was to find neural correlates of surprise and in 
# particular of surprising events we consider "impossible". 
# The data used are beta estimate NIfTI images derived from a GLM using SPM12 
# in MATLAB. 

##########################
# PURPOSE OF THIS SCRIPT #
##########################
# The purpose of this script is to look into the eyetracking data and check 
# if subjects fixated at the same position during the special moment pre vs post
# revelation

################################
# FUNCTIONALITY OF THIS SCRIPT #
################################
# Additionally move the edf files to sourcedata
# FIRST STEP 
# Import all libraries needed and get all important path information
# EXTRA STEP
# Needs to be done once !!!
# Copy the ascii files from the 'derivatives/EyeTracking' directory to 'rawdata'
# In that step change the name of the file from s<subject>b<block>r<run>.asc to
# sub-<subject>_task-magic_run-<run>_recording-eyetracking_physio.asc
# (maybe add a json file)
# SECOND STEP 
# Read in the info_MagicMoment.mat file created by author VP that contains the
# specific frame of the special moment.
# Read in the event.tsv files and the eyetracking ascii files 
# From the event files get the video presented, select the coresponding timepoint
# from the mat file and the fixation
# THIRD STEP
# Pool the fixation points of the magic moment +- some slack time for each subject
# and trial. 
# FOURTH STEP
# Do some plotting

In [2]:
# FIRST STEP
# interact with the operating system 
import os
from pathlib import Path
import shutil
import git
import glob
# import/export data
import h5py
# data structuration and calculations
import pandas as pd  # to create data frames
import numpy as np   # most important numerical calculations
# read in mat files
import readmat
# needed to extract the run number out of the parentesis of the string in the SPM.mat file
import re
# optimize time performance
import time
# plotting
import matplotlib.pyplot as plt

In [12]:
################################################
# VARIABLES FOR PATH SELECTION AND DATA ACCESS #
################################################
HOME            = str(Path.home())
PROJ_DIR        = os.path.join(HOME, 'Documents/Magic_fMRI/DATA/MRI')
RAW_DIR         = os.path.join(PROJ_DIR, 'rawdata')
DERIVATIVES_DIR = os.path.join(PROJ_DIR, 'derivatives')
ET_DIR = os.path.join(DERIVATIVES_DIR, 'eyetracking')
SUBJECS = glob.glob(os.path.join(RAW_DIR, 'sub*'))
SUBJECS.sort()

In [16]:
ET_SUBS

['/gpfs01/bartels/user/vplikat/Documents/Magic_fMRI/DATA/MRI/derivatives/eyetracking/sub-07']

In [15]:
# EXTRA STEP (if completed move down to the end and somehow document the step)
ET_SUBS = glob.glob(os.path.join(ET_DIR,'sub-07'))
ET_SUBS.sort()
for s, sub in enumerate(ET_SUBS):
    # get just the name of current subject without fullpath
    sub_name = os.path.basename(sub)
    # get all .asc files and move them (renamed) to raw
    asc_files = glob.glob(os.path.join(sub,'*.asc'))
    asc_files.sort()
    for file in asc_files:
        # ascii file look like that s1b2r5.asc
        # s=subject (1-24) b=block (1-3) r=run (1-5)
        # each block has five runs and shows videos of one object
        # runs 1&2 are pre revelation runs, run 3 is the revelation, runs 4&5 are post revelation runs
        # get subject, block and run index (sub index actually not important)
        indices = list(map(int, re.findall(r'\d+', os.path.basename(file))))
        # if current file is one of the pre revelation runs, add four per passed block plus run index
        if indices[2]<3:
            run_index = (indices[1]-1)*4 + indices[2]
        # else current file is one of the post revelation runs, add four per passed block plus run index -1 because in between there was the revelation run
        else:
            run_index = (indices[1]-1)*4 + indices[2]-1
            
        # sub-<subindex>...run-<runindex>
        file_destination_name = 'sub-{:02d}_task-magic_run-{:02d}_recording-eyetracking_physio.asc'.format(indices[0], run_index)
        dest_dir = os.path.join(RAW_DIR,'sub-{:02d}'.format(indices[0]),'func',file_destination_name)
        # copy file to rawdata and rename it to file_destination_name
        shutil.copy2(file, dest_dir)